In [33]:
import pandas as pd
import glob
import numpy as np

In [2]:
file_list = glob.glob('./Data/*.csv')
for file_name in file_list:
    print(file_name)

./Data/BGCON_CUST_DATA.csv
./Data/BGCON_CLAIM_DATA.csv
./Data/BGCON_FPINFO_DATA.csv
./Data/BGCON_FMLY_DATA.csv
./Data/BGCON_CNTT_DATA.csv


In [3]:
# read BGCON_CUST_DATA.csv ( 고객정보 )
customer_df = pd.read_csv(file_list[0], encoding='utf-16')

# read BGCON_CLAIM_DATA.csv ( 지급정보 )
claim_df = pd.read_csv(file_list[1], encoding='utf-16')

# read BGCON_FPINFO_DATA.csv ( 설계사정보 )
fpinfo_df = pd.read_csv(file_list[2], encoding='utf-16')

# read BGCON_FMLY_DATA.csv ( 가족정보 )
family_df = pd.read_csv(file_list[3], encoding='utf-16')

# read BGCON_CNTT_DATA.csv ( 계약정보 )
contract_df = pd.read_csv(file_list[4], encoding='utf-16')

In [4]:
# < WORKFLOW >
# Join : cust + cntt (on CUST_ID)
# Join : cntt + claim (on POLY_NO)
# Join : cntt + fpinfo (on CLLT_FP_PRNO)
# Create Feature : fmly (ex == 0 or 1 + 관계코드)

In [5]:
# Suggestion : 변수들의 조합으로 라는 말은 즉 새로운 feature를 만들어라.
#            : 외부데이터를 연계하여 활용해보아라 ( 예를 들어, 지도 api를 활용할 수 있지않을까? )

# Next Step 
# 먼저, Nan 을 어떻게 처리할 것인지? ( drop or interpolation )
# 어떤 알고리즘을 이용하여 예측할것인지 ( SVM ...? )

In [6]:
# Join : cust + cntt (on CUST_ID)
merged_df0 = pd.merge(customer_df, contract_df, on='CUST_ID')

# Join : cntt + claim (on POLY_NO)
merged_df1 = pd.merge(merged_df0, claim_df, on='POLY_NO')

# Join : cntt + fpinfo (on CLLT_FP_PRNO)
merged_df2 = pd.merge(merged_df1, fpinfo_df, on = 'CLLT_FP_PRNO')

In [7]:
merged_df2.tail()

,CUST_ID_x,DIVIDED_SET,SIU_CUST_YN,SEX,AGE,RESI_COST,RESI_TYPE_CODE,FP_CAREER,CUST_RGST,CTPR,...,COUNT_TRMT_ITEM,DCAF_CMPS_XCPA,NON_PAY_RATIO,HEED_HOSP_YN,INCB_DVSN,ETRS_YM,FIRE_YM,BEFO_JOB,EDGB,BRCH_CODE
119987,21483,1,N,2,30,31944,20,N,200306,경기,...,NaN,NaN,0,N,P,199711,999912,무직,대학졸,78
119988,20356,1,N,1,15,27777,20,N,200306,경기,...,NaN,NaN,0,N,R,200006,200403,무직,고졸,120
119989,20356,1,N,1,15,27777,20,N,200306,경기,...,NaN,NaN,0,N,R,200006,200403,무직,고졸,120
119990,17118,1,N,2,18,21527,20,N,200306,경기,...,NaN,NaN,0,N,R,198108,200206,기타,고졸,19
119991,17118,1,N,2,18,21527,20,N,200306,경기,...,NaN,NaN,0,N,R,198108,200206,기타,고졸,19


## Merge Family df ==> #Not complete

In [27]:
family_df['CUST_ID'].unique().shape

(376,)

In [99]:
is_cus = family_df['CUST_ID'] == 9790
family_df[is_cus]

,CUST_ID,SUB_CUST_ID,FMLY_RELN_CODE
11,9790,19390,14
88,9790,19317,14
220,9790,10857,14
294,9790,12957,17


In [85]:
is_sub = family_df['SUB_CUST_ID'] == 9790
family_df[is_sub]

,CUST_ID,SUB_CUST_ID,FMLY_RELN_CODE
123,12957,9790,17
174,10857,9790,13
233,19317,9790,13
399,19390,9790,13


In [92]:
# 지금하려는것 family_df 에 있는 value 를 새로운 column으로 추가 시키기 ==> family_df join 문제 해결
# 본인(11) 은 family_df에 아무도 없음 ==> 가족관계가 명시되어 있지 않은 사람은 11
# 가족 수 추가...? 아무도없음(0) / 1명 / 2명 
# 의문점 : 6191 처럼 가족이 두명인 애들이 있으므로 가족수를 추가해야할까? 
# 아니면, 11 / 14 / 13 /  +  0 / 2 / 2 ( 본인제외 가족 수 )
# 의문점 : 각 고객 마다 관계코드가 다른데 그걸 표현할수 있는 방법은...?
# ==> 그냥 함께 가입된 가족수만을 기입하는것은 쉬울듯하다... 
# ==> 그렇지만 만약 다른 테이블에 가족번호가 필요한 경우가 있다면, 즉 어느가족그룹인지 고객끼리 어떤 관계에 있는지가 필요하다면...?
# 음.. 가족수 + 가족마다의 그룹 별 번호를 새로 지정하여 ... 피쳐로 적용한다면...?
# ex) 가족유무 0 or 1 + 가족그룹번호 0~N + 가족수 0~5

In [38]:
# family df 에 id가 있으면 fmly_reln_code 를 추가 하고, 가족수를 체크하여 표기함.
# 그렇지 않으면 본인(11) 으로 추가, 가족수도 0으로 표기함. == filna에서 처리해줘도 될듯함.

In [89]:
customers_having_family = family_df['CUST_ID'].unique()

In [90]:
is_cus = family_df['CUST_ID'] == 9790
relevant_code = family_df[is_cus]['FMLY_RELN_CODE'].tolist()
numbers_of_family = len(family_df[is_cus]['FMLY_RELN_CODE'])

relevant_code, numbers_of_family

([14, 14, 14, 17], 4)

In [160]:
# for customer in customers_having_family:
#     is_cus = family_df['CUST_ID'] == customer
#     relevant_code = family_df[is_cus]['FMLY_RELN_CODE'].tolist()[0]
#     numbers_of_family = len(family_df[is_cus]['FMLY_RELN_CODE'])

#     print(relevant_code, numbers_of_family)